In [1]:
import numpy as np
import random
import struct
import warnings
import os
# Suppress RuntimeWarnings related to overflow
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [12]:
def float_to_bin(num):
    '''Input floating points. Output 32 bits single-percision string in binary'''
    bits, = struct.unpack('!I', struct.pack('!f', num))
    return "{:032b}".format(bits)

def tf32_to_bin(num):
    '''Input floating point. Output first 19 bits (MSB) of 32-bit single-precision binary string'''
    bits, = struct.unpack('!I', struct.pack('!f', num))
    full_bin = "{:032b}".format(bits)
    return full_bin[:19]
def binStr_to_hex(bin_str):
    """Input a binary string. Output a HEX string."""
    # Convert binary string to an integer
    int_value = int(bin_str, 2)
    
    # Convert integer to hexadecimal and remove the '0x' prefix
    hex_value = hex(int_value)[2:]
    
    # Pad with leading zeros to ensure it's 8 characters long (32 bits)
    hex_value = hex_value.zfill(8)  # Ensure it's 8 hex digits (32 bits)
    
    return hex_value
def binStr_to_float32(bin_str):
    """Input a 32 bit binary string. Output a np.float32 number."""
    if len(bin_str) != 32:
        raise ValueError("The binary string must be 32 bits long.")
    # Convert the binary string to an unsigned 32-bit integer
    int_representation = int(bin_str, 2)
    # Convert the integer to a float32
    float_representation = np.float32(np.frombuffer(np.uint32(int_representation).tobytes(), dtype=np.float32)[0])
    return float_representation

RANDOMLY GENERATE TEST PATTERN( include some corner cases)

In [3]:
def generate_random_uniform(MIN, MAX):
    """randomly generate float32 number in uniform distribution"""
    random_pos = np.random.uniform(np.float32(MIN), np.float32(MAX))
    random_sign = random.choice([-1, 1])
    random_float32 = np.float32((random_pos * random_sign))
    return np.float32(random_float32)

In [4]:
def generate_random_log(MIN, MAX):
    """randomly generate float32 number in log distribution"""
    # Constants representing the range limits in log space
    min_log = np.log(MIN).astype(np.float32)
    max_log = np.log(MAX).astype(np.float32)
    # Generate a random float32 in log space, then exponentiate it
    random_log_value = np.random.uniform(min_log, max_log)
    random_pos = np.exp(random_log_value).astype(np.float32)
    random_sign = random.choice([-1, 1])
    random_float32 = np.float32((random_pos * random_sign))
    return np.float32(random_float32)

In [5]:
def generate_random_pattern_list(INST='ADD',  UNIFORM_NORM_NUM=0, LOG_NORM_NUM=0):
    
    """RANDOMLY GENERATE PATTERN LIST"""
    # lists of np.float32 numbers
    pattern_A_list = []
    pattern_B_list = []
    pattern_OUT_list = []

    # randomly generate 2 NORMAL numbers in UNIFORM distribution
    for _ in range(UNIFORM_NORM_NUM):
        A = generate_random_uniform(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        B = generate_random_uniform(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false
    # randomly generate 2 NORMAL numbers in LOG distribution
    for _ in range(LOG_NORM_NUM):
        A = generate_random_log(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        B = generate_random_log(MIN = 2**-126, MAX = (2 - 2**-23) * 2**127)
        pattern_A_list.append(A)
        pattern_B_list.append(B)
        if INST == 'ADD':
            pattern_OUT_list.append(A + B)
        elif INST == 'SUB':
            pattern_OUT_list.append(A - B)
        elif INST == 'CMP':
                pattern_OUT_list.append(np.float32(A < B)) #1: true, 0: false
    

    return pattern_A_list, pattern_B_list, pattern_OUT_list

WRITE OUTPUT DATA

In [17]:
# Function to write pairs into a text file
def write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir = '',  file_dir_A='A.dat', file_dir_B='B.dat', file_dir_OUT='OUT.dat'):
    # A
    with open(os.path.join(root_dir, file_dir_A), 'w') as file:
        for num in pattern_A_list:
            file.write(f"{tf32_to_bin(num)}\n")
    # B
    with open(os.path.join(root_dir, file_dir_B), 'w') as file:
        for num in pattern_B_list:
            file.write(f"{tf32_to_bin(num)}\n")
    # OUT
    # if INST == 'CMP':
    #     with open(os.path.join(root_dir, file_dir_OUT), 'w') as file:
    #         for num in pattern_OUT_list:
    #             if num == 1: #TRUE  
    #                 file.write(f"00000000000000000000000000000001 // TRUE\n")
    #             elif num == 0: #FALSE  
    #                 file.write(f"00000000000000000000000000000000 // FALSE\n")
    # elif INST == 'ADD' or INST == 'SUB':
    #     with open(os.path.join(root_dir, file_dir_OUT), 'w') as file:
    #         for num in pattern_OUT_list:
    #             file.write(f"{tf32_to_bin(num)}  // {binStr_to_hex(float_to_bin(num))}\n")

MAIN

In [18]:
UNIFORM_NORM_NUM = 1000
LOG_NORM_NUM = 1000


root_dir = "./MY_pattern/ADD"
INST='ADD'
pattern_A_list, pattern_B_list, pattern_OUT_list = generate_random_pattern_list(INST, UNIFORM_NORM_NUM, LOG_NORM_NUM)
write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir)
print("ADD pattern complete")

# root_dir = "./MY_pattern/SUB"
# INST='SUB'
# pattern_A_list, pattern_B_list, pattern_OUT_list = generate_random_pattern_list(INST, UNIFORM_NORM_NUM, LOG_NORM_NUM)
# write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir)
# print("SUB pattern complete")

# root_dir = "./MY_pattern/CMP"
# INST='CMP'
# pattern_A_list, pattern_B_list, pattern_OUT_list = generate_random_pattern_list(INST, UNIFORM_NORM_NUM, LOG_NORM_NUM)
# write_to_txt(INST, pattern_A_list, pattern_B_list, pattern_OUT_list, root_dir)
# print("CMP pattern complete")

ADD pattern complete
